In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable



In [ ]:
#!pip install pyspark --break-system-packages
#!pip install delta-spark==2.1.1 --break-system-package

In [2]:
import os
from pyspark import SparkContext, SparkConf
from pyspark import SQLContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext;
from pyspark.sql.session import SparkSession
from delta import *


conf = SparkConf().setAppName('app').setMaster('local[*]').setAll([("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"),
                                                                   ("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension"),
                                                                   ("spark.sql.execution.arrow.pyspark.enabled", "true"), 
                                                                   ("spark.memory.fraction", 0.8) ,("spark.executor.memory", "14g"),
                                                                   ("spark.driver.memory", "12g"),("spark.sql.shuffle.partitions" , "800"),
                                                                   ('spark.driver.extraClassPath', '/usr/share/java/mssql-jdbc-12.8.1.jre8.jar'),
                                                                   ('spark.executor.extraClassPath', '/usr/share/java/mssql-jdbc-12.8.1.jre8.jar'),
                                                                  ("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0")])

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)


25/03/21 10:24:02 WARN Utils: Your hostname, DESKTOP-2VPR9IB resolves to a loopback address: 127.0.1.1; using 172.30.143.1 instead (on interface eth0)
25/03/21 10:24:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/usr/local/lib/python3.12/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-abdfdb75-20fb-4d8b-b359-be0c1dad4303;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 155ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|

25/03/21 10:24:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
#%%time
#data.load().write.mode('overwrite').option("header",True) \
# .csv("/home/rajavel/spark")

In [3]:
from delta.tables import *
#df = sqlContext.read.format("delta").load("/home/rajavel/spark/delta/events")
deltaTable = DeltaTable.forPath(sqlContext.sparkSession, "/home/rajavel/spark/delta/events")
#df.show()

In [4]:
data = sqlContext.read \
     .format('jdbc') \
     .option('url', 'jdbc:sqlserver://127.0.0.1:1433;encrypt=true;trustServerCertificate=true') \
     .option("driver", 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
     .option('user', 'sa') \
     .option('password', '####') \
     .option('query', '(SELECT * FROM AdventureWorksDW2022.dbo.DimCustomer)')

#data.load().write.format("delta").mode("overwrite").save("/home/rajavel/spark/delta/events");

In [8]:
deltaTable.alias("oldData") \
  .merge(
    data.load().alias("newData"),
    "oldData.CustomerKey = newData.CustomerKey") \
  .whenMatchedUpdate(set = { "CustomerKey": col("newData.CustomerKey") }) \
  .whenNotMatchedInsert(values = { "CustomerKey": col("newData.CustomerKey") }) \
  .execute()

25/03/21 10:25:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
deltaTable.toDF().count()

18486

In [13]:
df = sqlContext.read.format("delta") \
  .option("versionAsOf", 0) \
  .load("/home/rajavel/spark/delta/events")

df.count()

18485

In [ ]:
file_location = "/tmp/customers.csv"
file_type = "csv"

infer_schema = "true"
first_row_is_header = "true"
delimiter = "\t"
max_cols = 15000000 # 14848614 variants loaded

#data = sqlContext.read.format(file_type) \
#  .option("inferSchema", infer_schema) \
#  .option("header", first_row_is_header) \
#  .option("sep", delimiter) \
#  .option("maxColumns", max_cols) \
#  .load(file_location)

In [12]:
stream2 = sqlContext.readStream.format("delta") \
  .load("/home/rajavel/spark/delta/events") \
  .writeStream.format("console") \
  .start()

25/03/21 10:30:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-786ec3e4-663d-41cf-918d-15cf1c3d62be. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/21 10:30:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+------------+--------------------+-----+---------+----------+--------+---------+----------+-------------+------+------+--------------------+------------+-------------+--------------------+-------------------+--------------------+---------------+-----------------+--------------------+----------------+--------------+---------------+--------------------+--------

In [17]:
fullHistoryDF = deltaTable.history()

In [20]:
fullHistoryDF.show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2025-03-21 10:25:...|  null|    null|    MERGE|{predicate -> (ol...|null|    null|     null|          0|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.3....|
|      0|2025-03-21 09:56:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|       null|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.2....|
+-------+-